### Project 6 - Solution

```
1. Supplier exceptions
    a. Not manufactured anymore
    b. Production delayed
    c. Shipping delayed
    
2. Checkout exceptions
    a. Inventory type exceptions
        - out of stock
    b. Pricing exceptions
        - invalid coupon code
        - cannot stack coupons
```

In [1]:
from datetime import datetime

In [2]:
class WidgetException(Exception):
    message = 'Generic Widget exception.'

    def __init__(self, *args, customer_message=None):
        super().__init__(args)
        if args:
            self.message = args[0]
        self.customer_message = customer_message if customer_message is not None else self.message

    def log_exception(self):
        exception = {
            "type": type(self).__name__,
            "message": self.message,
            "args": self.args[1:]
        }
        print(f'EXCEPTION: {datetime.now().isoformat()}: {exception}')

In [3]:
ex1 = WidgetException('some custome message', 10, 100)
ex2 = WidgetException(customer_message='Custom user message')

In [4]:
ex1.args

(('some custome message', 10, 100),)

In [5]:
ex2.args

((),)

In [6]:
ex1.log_exception()

EXCEPTION: 2026-02-03T14:16:11.807799: {'type': 'WidgetException', 'message': 'some custome message', 'args': ()}


In [8]:
ex2.log_exception()

EXCEPTION: 2026-02-03T14:16:21.186789: {'type': 'WidgetException', 'message': 'Generic Widget exception.', 'args': ()}


In [9]:
class SupplierException(WidgetException):
    message = 'Supplier exception'

class NotManufacturedException(SupplierException):
    message = 'Widget is no longer manufactured by supplier'

class ProductionDelayedException(SupplierException):
    message = 'Widget production has been delayed by manufacturer'

class ShippingDelayedException(SupplierException):
    message = 'Widget shipping has been delayed by supplier'

class CheckoutException(WidgetException):
    message = 'Checkout exception'

class InventoryException(CheckoutException):
    message = 'Checkout inventory exception'

class OutOfStockException(InventoryException):
    message = 'Inventory out of stock'

class PricingException(CheckoutException):
    message = 'Chekcout pricing exception'

class InvalidCouponCodeException(PricingException):
    message = 'Invalid checkout coupon code'

class CannotStackCouponException(PricingException):
    message = 'Cannot stack checkout coupon code'

In [10]:
try:
    raise CannotStackCouponException()
except CannotStackCouponException as ex:
    ex.log_exception()
    raise

EXCEPTION: 2026-02-03T14:20:54.331786: {'type': 'CannotStackCouponException', 'message': 'Cannot stack checkout coupon code', 'args': ()}


CannotStackCouponException: ()

In [12]:
from datetime import datetime
from http import HTTPStatus
import json

In [13]:
class WidgetException(Exception):
    message = 'Generic Widget exception.'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR

    def __init__(self, *args, customer_message=None):
        super().__init__(args)
        if args:
            self.message = args[0]
        self.customer_message = customer_message if customer_message is not None else self.message

    def log_exception(self):
        exception = {
            "type": type(self).__name__,
            "message": self.message,
            "args": self.args[1:]
        }
        print(f'EXCEPTION: {datetime.now().isoformat()}: {exception}')

    def to_json(self):
        response = {
            'code': self.http_status.value,
            'message': '{}: {}'.format(self.http_status.phrase, self.customer_message),
            'category': type(self).__name__,
            'time_utc': datetime.now().isoformat()
        }
        return json.dumps(response)

In [14]:
e = WidgetException('same custom message for log and user')

In [15]:
e.log_exception()

EXCEPTION: 2026-02-03T14:25:12.826258: {'type': 'WidgetException', 'message': 'same custom message for log and user', 'args': ()}


In [17]:
json.loads(e.to_json())

{'code': 500,
 'message': 'Internal Server Error: same custom message for log and user',
 'category': 'WidgetException',
 'time_utc': '2026-02-03T14:25:33.868127'}

In [18]:
e = WidgetException('custom internal message', customer_message='custom user message')

In [19]:
e.log_exception()

EXCEPTION: 2026-02-03T14:26:44.205017: {'type': 'WidgetException', 'message': 'custom internal message', 'args': ()}


In [21]:
json.loads(e.to_json())

{'code': 500,
 'message': 'Internal Server Error: custom user message',
 'category': 'WidgetException',
 'time_utc': '2026-02-03T14:27:00.232564'}

In [23]:
try:
    raise WidgetException('custom error message')
except WidgetException as ex:
    print(repr(ex.__traceback__))

In [24]:
import traceback

In [27]:
try:
    raise ValueError()
except ValueError:
    try:
        raise WidgetException('custom message')
    except WidgetException as ex:
        tb = traceback.TracebackException.from_exception(ex)
        print(list(tb.format()))

['Traceback (most recent call last):\n', '  File "/tmp/ipykernel_1533/1583212970.py", line 2, in <module>\n    raise ValueError()\n', 'ValueError\n', '\nDuring handling of the above exception, another exception occurred:\n\n', 'Traceback (most recent call last):\n', '  File "/tmp/ipykernel_1533/1583212970.py", line 5, in <module>\n    raise WidgetException(\'custom message\')\n', "WidgetException: ('custom message',)\n"]


In [28]:
try:
    raise ValueError()
except ValueError:
    try:
        raise WidgetException('custom message')
    except WidgetException as ex:
        tb = traceback.TracebackException.from_exception(ex)
        tb = ''.join(tb.format())
        print(tb)

Traceback (most recent call last):
  File "/tmp/ipykernel_1533/563563182.py", line 2, in <module>
    raise ValueError()
ValueError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipykernel_1533/563563182.py", line 5, in <module>
    raise WidgetException('custom message')
WidgetException: ('custom message',)



In [ ]:
class WidgetException(Exception):
    message = 'Generic Widget exception.'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR

    def __init__(self, *args, customer_message=None):
        super().__init__(*args)
        if args:
            self.message = args[0]
        self.customer_message = customer_message if customer_message is not None else self.message

    @property
    def traceback(self):
        return traceback.TracebackException.from_exception(self).format()

    def log_exception(self):
        exception = {
            "type": type(self).__name__,
            "message": self.message,
            "args": self.args[1:],
            "traceback": list(self.traceback)
        }
        print(f'EXCEPTION: {datetime.now().isoformat()}: {exception}')

    def to_json(self):
        response = {
            'code': self.http_status.value,
            'message': '{}: {}'.format(self.http_status.phrase, self.customer_message),
            'category': type(self).__name__,
            'time_utc': datetime.now().isoformat()
        }
        return json.dumps(response)

In [ ]:
try:
    raise WidgetException()
except WidgetException as ex:
    ex.log_exception()
    print('---------------')
    print(ex.to_json())


EXCEPTION: 2026-02-03T14:41:28.915956: {'type': 'WidgetException', 'message': 'Generic Widget exception.', 'args': (), 'traceback': ['Traceback (most recent call last):\n', '  File "/tmp/ipykernel_1533/162776454.py", line 2, in <module>\n    raise WidgetException()\n', 'WidgetException: ()\n']}
---------------
{"code": 500, "message": "Internal Server Error: Generic Widget exception.", "category": "WidgetException", "time_utc": "2026-02-03T14:41:28.916147"}


In [31]:
try:
    a = 1 / 0
except ZeroDivisionError:
    try:
        raise WidgetException()
    except WidgetException as ex:
        print(''.join(ex.traceback))

Traceback (most recent call last):
  File "/tmp/ipykernel_1533/1535497075.py", line 2, in <module>
    a = 1 / 0
        ~~^~~
ZeroDivisionError: division by zero

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipykernel_1533/1535497075.py", line 5, in <module>
    raise WidgetException()
WidgetException: ()



In [34]:
class SupplierException(WidgetException):
    message = 'Supplier exception'

class NotManufacturedException(SupplierException):
    message = 'Widget is no longer manufactured by supplier'

class ProductionDelayedException(SupplierException):
    message = 'Widget production has been delayed by manufacturer'

class ShippingDelayedException(SupplierException):
    message = 'Widget shipping has been delayed by supplier'

class CheckoutException(WidgetException):
    message = 'Checkout exception'

class InventoryException(CheckoutException):
    message = 'Checkout inventory exception'

class OutOfStockException(InventoryException):
    message = 'Inventory out of stock'

class PricingException(CheckoutException):
    message = 'Chekcout pricing exception'

class InvalidCouponCodeException(PricingException):
    message = 'Invalid checkout coupon code'

class CannotStackCouponException(PricingException):
    message = 'Cannot stack checkout coupon code'

In [36]:
e = InvalidCouponCodeException('User tried to pull a fast one on us.', customer_message='Sorry. This coupon is not valid.')

In [37]:
e.log_exception()

EXCEPTION: 2026-02-03T14:43:45.816459: {'type': 'InvalidCouponCodeException', 'message': 'User tried to pull a fast one on us.', 'args': (), 'traceback': ["InvalidCouponCodeException: ('User tried to pull a fast one on us.',)\n"]}


In [38]:
e.to_json()

'{"code": 500, "message": "Internal Server Error: Sorry. This coupon is not valid.", "category": "InvalidCouponCodeException", "time_utc": "2026-02-03T14:44:10.082233"}'

In [40]:
try:
    raise ValueError()
except ValueError:
    try:
        raise InvalidCouponCodeException(
            'User tried to pull a fast one on us.',
            customer_message='Sorry. This coupon is not valid.'
        )
    except InvalidCouponCodeException as ex:
        ex.log_exception()
        print('-------------')
        print(ex.to_json())
        print('-----------------')
        print(''.join(ex.traceback))

EXCEPTION: 2026-02-03T14:45:42.712766: {'type': 'InvalidCouponCodeException', 'message': 'User tried to pull a fast one on us.', 'args': (), 'traceback': ['Traceback (most recent call last):\n', '  File "/tmp/ipykernel_1533/1435695803.py", line 2, in <module>\n    raise ValueError()\n', 'ValueError\n', '\nDuring handling of the above exception, another exception occurred:\n\n', 'Traceback (most recent call last):\n', '  File "/tmp/ipykernel_1533/1435695803.py", line 5, in <module>\n    raise InvalidCouponCodeException(\n', "InvalidCouponCodeException: ('User tried to pull a fast one on us.',)\n"]}
-------------
{"code": 500, "message": "Internal Server Error: Sorry. This coupon is not valid.", "category": "InvalidCouponCodeException", "time_utc": "2026-02-03T14:45:42.713522"}
-----------------
Traceback (most recent call last):
  File "/tmp/ipykernel_1533/1435695803.py", line 2, in <module>
    raise ValueError()
ValueError

During handling of the above exception, another exception occ